### Fine-tune XLM-R for Hallucination Classification

In [8]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers --quiet
!pip install scikit-learn --quiet

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

### Load tokenizer + model (XLM-R-base)

In [9]:
MODEL_NAME = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=3, 
    id2label={0: "no", 1: "extrinsic", 2: "intrinsic"},
    label2id={"no": 0, "extrinsic": 1, "intrinsic": 2}
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Dataset class đọc .jsonl

In [10]:
import json

class JsonlDataset(Dataset):
    def __init__(self, path, tokenizer, max_len=256):
        self.samples = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                self.samples.append(json.loads(line))
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Ghép các message thành 1 đoạn text duy nhất
        text = " ".join([m["content"] for m in sample["messages"]])
        
        # Ánh xạ label từ text -> số
        label_map = {"no": 0, "extrinsic": 1, "intrinsic": 2}
        label = label_map[sample["label"]]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

### Load dữ liệu train/val/test

In [11]:
train_dataset = JsonlDataset("../train.jsonl", tokenizer)
val_dataset   = JsonlDataset("../val.jsonl", tokenizer)
test_dataset  = JsonlDataset("../test.jsonl", tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)

### Cấu hình train

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

num_training_steps = num_epochs * len(train_loader)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

### Training loop

In [13]:
from tqdm import trange

save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")
    
    model.train()
    total_loss = 0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training", leave=True)
    for step, batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        avg_loss = total_loss / (step + 1)
        loop.set_postfix(loss=loss.item(), avg_loss=avg_loss)

    avg_train_loss = total_loss / len(train_loader)
    print(f"Average training loss for epoch {epoch+1}: {avg_train_loss:.4f}")

    # Lưu checkpoint cuối mỗi epoch
    epoch_save_path = os.path.join(save_dir, f"epoch_{epoch+1}")
    os.makedirs(epoch_save_path, exist_ok=True)
    model.save_pretrained(epoch_save_path)
    tokenizer.save_pretrained(epoch_save_path)
    print(f"Saved checkpoint to {epoch_save_path}")


===== Epoch 1/3 =====


Training: 100%|██████████| 10/10 [03:18<00:00, 19.90s/it, avg_loss=1.13, loss=1.09]


Average training loss for epoch 1: 1.1252
Saved checkpoint to ./checkpoints\epoch_1

===== Epoch 2/3 =====


Training: 100%|██████████| 10/10 [02:05<00:00, 12.60s/it, avg_loss=1.1, loss=1.11]


Average training loss for epoch 2: 1.0987
Saved checkpoint to ./checkpoints\epoch_2

===== Epoch 3/3 =====


Training: 100%|██████████| 10/10 [02:28<00:00, 14.90s/it, avg_loss=1.1, loss=1.1] 


Average training loss for epoch 3: 1.1000
Saved checkpoint to ./checkpoints\epoch_3


In [14]:
SAVE_DIR = "./model"

tokenizer.save_pretrained(SAVE_DIR)
model.save_pretrained(SAVE_DIR)
print(f"Saved final model to {SAVE_DIR}")

Saved final model to ./model
